# Denison CS181/DA210 Homework 2.g - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [ ]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

---

## Part A: Data normalization

**Q1:** We first consider a dataset based on a tuburculosis dataset from the World Health Organization's ["Global Tuberculosis Report"](https://www.int/tb/country/data/download/en/).

At no point should you rearrange the order of the rows.

To start, read `table6.csv` into a dataframe `df1`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display the untidy DataFrame
df1

In [ ]:
# Testing cell
assert df1.shape == (6,4)
assert df1.iloc[2,3] == '37737/172006362'

**Q2:** We'll now make this data tidy, using the following steps:

1. Combine `century` and `yearDigits` into one column, `year` (whose values are strings), then drop the two old columns. Use `copy()` to avoid modifying the original data frame. Store the result as `df1a`.
2. Starting from `df1a`, split the column `rate` into two new columns: `cases` (the number before the slash) and `population` (the number after). After you're done, drop `rate`. Store the result as `df1b`.  Again, use `copy()` to avoid modifying either `df1` or `df1a`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display the tidy DataFrame
df1b

In [ ]:
# Testing cell
assert df1.shape == (6,4)
assert df1a.shape == (6,3)
assert df1b.shape == (6,4)
assert df1.iloc[2,3] == '37737/172006362'
assert df1a.iloc[3,2] == '2000'
assert df1b.iloc[4,3] == '1272915272'
assert df1b.iloc[0,2] == "745"

---

**Q3:** Read `us_rent_income.csv` into a dataframe (with `GEOID` as the index) named `df_rent_untidy`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display part of the untidy DataFrame
df_rent_untidy.head()

In [ ]:
# Testing cell
assert df_rent_untidy.shape == (104,5)
assert df_rent_untidy.iloc[0,2] == "income"
assert df_rent_untidy.iloc[1,3] == 747.0

**Q4:** Transform `df_rent_untidy` as needed to make it tidy. Store the result as `df_rent`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display part of the tidy DataFrame
df_rent.head()

In [ ]:
# Testing cell
assert df_rent.shape == (52,5)
assert df_rent.iloc[0,1] == 24476.0
assert df_rent.iloc[0,2] == 747.0
assert df_rent.iloc[0,3] == 136.0
assert df_rent.iloc[0,4] == 3.0
assert df_rent.iloc[20,1] == 37147.0
assert df_rent.iloc[31,2] == 809.0

---

**Q5:** Consider the data on religions and income, gathered by Pew Research Center and hosted at this link: https://github.com/chendaniely/pandas_for_everyone/blob/master/data/pew.csv.

The data is also available as `"pew.csv"` in the data folder.  In the code cell that follows, read the data into a DataFrame assigned to `df_rel_untidy`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display part of the DataFrame
df_rel_untidy.head()

In [ ]:
# Testing cell
assert df_rel_untidy.shape == (18,11)
assert df_rel_untidy.iloc[0,0] == "Agnostic"
assert df_rel_untidy.iloc[0,1] == 27
assert df_rel_untidy.iloc[5,0] == "Evangelical Prot"
assert df_rel_untidy.columns[5] == "$40-50k"
assert df_rel_untidy.iloc[14,8] == 14

**Q6:** This DataFrame is not in tidy form.  Give one of the three tidy-data requirements that it fails to satisfy and explain why.

YOUR ANSWER HERE

**Q7:** Explore the data from the previous question, then **from the data** list the independent variable(s) and the dependent variable(s). Note: this data came from a survey of counting individuals based on their religion and their income category.

YOUR ANSWER HERE

**Q8:** Transform as needed to make it tidy.  Store the normalized (tidy) data frame as `df_rel`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display part of the tidy DataFrame
df_rel.head()

In [ ]:
# Testing cell
assert df_rel.shape == (180,3)
assert df_rel.iloc[0,0] == "Agnostic"
assert df_rel.iloc[0,1] == "<$10k"
assert df_rel.iloc[0,2] == 27
assert df_rel.iloc[41,0] == "Evangelical Prot"
assert df_rel.iloc[89,1] == "$40-50k"
assert df_rel.iloc[104,2] == 14

---

---

## Part B

**Q9:** How much time (in minutes/hours) did you spend on this homework assignment?

YOUR ANSWER HERE

**Q10:** Who was your partner for this assignment?  If you worked alone, say so instead.

YOUR ANSWER HERE